In [1]:
from sklearn.model_selection import LeavePOut
from sklearn.metrics import mean_squared_error
import networkx as nx
import math
import sys
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./OTCNet.csv')

N = 5

X =  df
#y =  df['rating']
X = X.values
#y = y.values

lpo = LeavePOut(N)
lpo.get_n_splits(X)

475837529743534588728

In [3]:
def initialize_scores(G):
    fairness = {}
    goodness = {}
    
    nodes = G.nodes()
    #print(nodes)
    for node in nodes:
        fairness[node] = 1
        try:
            goodness[node] = G.in_degree(node, weight='weight')*1.0/G.in_degree(node)
        except:
            goodness[node] = 0
    return fairness, goodness

In [4]:
def compute_fairness_goodness(G):
    fairness, goodness = initialize_scores(G)
    
    nodes = G.nodes()
    iter = 0
    while iter < 100:
        df = 0
        dg = 0
        for node in nodes:
            inedges = G.in_edges(node, data='weight')
            #print(inedges)
            g = 0
            for edge in inedges:
                g += fairness[edge[0]]*edge[2]

            try:
                dg += abs(g/len(inedges) - goodness[node])
                goodness[node] = g/len(inedges)
            except:
                pass
            
        for node in nodes:
            outedges = G.out_edges(node, data='weight')
            f = 0
            for edge in outedges:
                f += 1.0 - abs(edge[2] - goodness[edge[1]])/2.0
            try:
                df += abs(f/len(outedges) - fairness[node])
                fairness[node] = f/len(outedges)
            except:
                pass
        
        if df < math.pow(10, -6) and dg < math.pow(10, -6):
            break
        iter+=1
    
    return fairness, goodness

In [5]:
# only for first 20 iterations
itr = 10
fairness = {}
goodness = {}
predicted = pd.DataFrame(np.random.randn(5, 1), columns=['rat'])
test = pd.DataFrame(np.random.randn(5, 1), columns=['rat'])
goodness_test = {}
RMSE = 0
PCC = 0
for train_index, test_index in lpo.split(X):
    if test_index[4] < (itr+N):
        X_train, X_test = X[train_index], X[test_index]
        G = nx.DiGraph()
        test['rat'][0] = X_test[0][2]
        test['rat'][1] = X_test[1][2]
        test['rat'][2] = X_test[2][2]
        test['rat'][3] = X_test[3][2]
        test['rat'][4] = X_test[4][2]
        for index in range(0, 35587):
            G.add_edge(X_train[index][0], X_train[index][1], weight = float(X_train[index][2])) ## the weight should already be in the range of -1 to 1
        try:
            fairness, goodness = compute_fairness_goodness(G)
        except:
            pass
        try:
            predicted['rat'][0] = fairness[X_test[0][0]]*goodness[X_test[0][1]]
        except:
            predicted['rat'][0] = 0
            continue
        try:
            predicted['rat'][1] = fairness[X_test[1][0]]*goodness[X_test[1][1]]
        except:
            predicted['rat'][1] = 0
            continue
        try:
            predicted['rat'][2] = fairness[X_test[2][0]]*goodness[X_test[2][1]]
        except:
            predicted['rat'][2] = 0
            continue
        try:
            predicted['rat'][3] = fairness[X_test[3][0]]*goodness[X_test[3][1]]
        except:
            predicted['rat'][3] = 0
            continue
        try:
            predicted['rat'][4] = fairness[X_test[4][0]]*goodness[X_test[4][1]]
        except:
            predicted['rat'][4] = 0
            continue
    else:
        break

    RMSE += math.sqrt(mean_squared_error(test['rat'], predicted['rat']))
    PCC += np.corrcoef(test['rat'], predicted['rat'])
    print('iteration', test_index[4] - N)

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


In [6]:
RMES_fg_leaveNout = RMSE/itr
RMES_fg_leaveNout

0.406733391531918

In [7]:
PCC_fg_leaveNout = PCC/itr
PCC_fg_leaveNout

array([[ 1.        , -0.37802359],
       [-0.37802359,  1.        ]])